# Collecting and preparing data

In [1]:
import os, json, requests
import pandas as pd
import numpy as np
import cbsodata
from datetime import date, timedelta

with open('./data/sources.json', 'r') as json_file:
    resources = json.load(json_file)

def cleanup(df):
    for c in df.columns:
        if 'ersion' in c.lower():
            df.drop(columns=c, inplace=True)
    return df

## (Down)loading data

In [15]:
print('Pulling data from sources.json:')
for org,sets in resources.items():
    if org == 'CBS':
        for dataset,identifier in sets.items():
            name = '_'.join(dataset.lower().split())
            data_loc = './data/'+org.lower()+'_'+name+'.csv'
            try:
                os.stat(data_loc)
            except:
                pd.DataFrame(cbs.get_data(identifier)).to_csv(data_loc, index=False)
            finally:
                globals()[name] = pd.read_csv(data_loc)
            metadata = cbsodata.get_info(identifier)
            print('(Source: '+org+')  '+name+' @ '+'https://opendata.cbs.nl/statline/portal.html?_catalog=CBS&_la=nl&tableId='+identifier+'&_theme=76')
    else:
        for dataset,urls in sets.items():
            name = '_'.join(dataset.lower().split())
            data_loc = './data/'+org.lower()+'_'+name+'.csv'
            try:
                os.stat(data_loc)
            except:
                if urls['data'][-4:] == '.csv':
                    pd.read_csv(urls['data']).to_csv(data_loc, index=False)
                elif urls['data'][-5:] == '.json':
                    pd.read_json(urls['data']).to_csv(data_loc, index=False)
                else:
                    print('can not handle file format: just dopwnloading file')
                    data_loc = data_loc[-4:]+'.temp'
                    print(data_loc)
                    print(urls['data'])
                    req = requests.get(urls['data'], stream=True)
                    print(req)
                    with open(data_loc[:-5], 'wb') as f:
                        f.write(req.content)
            finally:
                globals()[name] = pd.read_csv(data_loc)
            print('(Source: '+org+') '+name+' @ '+urls['meta'])

Pulling data from sources.json:
(Source: CBS)  deceased @ https://opendata.cbs.nl/statline/portal.html?_catalog=CBS&_la=nl&tableId=70895ned&_theme=76
(Source: CBS)  population @ https://opendata.cbs.nl/statline/portal.html?_catalog=CBS&_la=nl&tableId=37230ned&_theme=76
(Source: RIVM) counts_per_municipality @ https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/5f6bc429-1596-490e-8618-1ed8fd768427
(Source: RIVM) prevalence @ https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/097155aa-75eb-4caa-8ed3-4c6edb80467e
(Source: RIVM) r @ https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/ed0699d1-c9d5-4436-8517-27eb993eab6e
(Source: RIVM) homes_for_elderly @ https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/0179dd26-7bf6-4021-857f-8623aaf8e73a
(Source: RIVM) homes_for_disabled @ https://data.rivm.nl/geonetwork/srv/dut/catalog.search#/metadata/aa28b74d-7150-47bc-a7cb-98876e93381b
(Source: RIVM) seventy_plus_living_at_home @ https://data.riv

## Preparing data
### RIVM

In [17]:
prevalence = cleanup(prevalence)
prevalence = prevalence.groupby('Date').sum()
prevalence.index = pd.PeriodIndex(prevalence.index, freq='D')
rivm = prevalence

counts_per_municipality.rename(columns={'Date_of_publication':'Date',
                                        'Total_reported': 'Positive_tests',
                                        'Deceased': 'Deceased_covid'
                                       }, inplace=True)
counts_per_municipality = counts_per_municipality.groupby('Date').sum()
counts_per_municipality = counts_per_municipality[['Deceased_covid', 'Positive_tests', 'Hospital_admission']]
counts_per_municipality.index = pd.PeriodIndex(counts_per_municipality.index, freq='D')
rivm = rivm.merge(counts_per_municipality, on='Date', how='outer')

hospital_admission.rename(columns={'Date_of_statistics': 'Date',
                                   'Hospital_admission_notification': 'Hospital_admission_NICE_notification',
                                   'Hospital_admission': 'Hospital_admission_NICE'
                                  }, inplace=True)
hospital_admission = cleanup(hospital_admission)
hospital_admission = hospital_admission.groupby('Date').sum()
hospital_admission = hospital_admission[['Hospital_admission_NICE','Hospital_admission_NICE_notification']]
hospital_admission.index = pd.PeriodIndex(hospital_admission.index, freq='D')
rivm = rivm.merge(hospital_admission, on='Date', how='outer')

ic_admission.rename(columns={'Date_of_statistics': 'Date'}, inplace=True)
ic_admission = cleanup(ic_admission)
ic_admission = ic_admission.groupby('Date').sum()
ic_admission = ic_admission[['IC_admission','IC_admission_notification']]
ic_admission.index = pd.PeriodIndex(ic_admission.index, freq='D')
rivm = rivm.merge(ic_admission, on='Date', how='outer')

hospital_occupation.rename(columns={'Datum': 'Date'}, inplace=True)
hospital_occupation['IC_Bedden_TOTAAL'] = hospital_occupation['IC_Bedden_COVID']+hospital_occupation['IC_Bedden_Non_COVID']
hospital_occupation = hospital_occupation.groupby('Date').sum()
hospital_occupation.index = pd.PeriodIndex(hospital_occupation.index, freq='D')
rivm = rivm.merge(hospital_occupation, on='Date', how='outer')

r = cleanup(r)
#r.rename(columns={'population': 'source_for_r'}, inplace=True)
r = r.groupby('Date').sum()
#r = r.groupby(['Date','source_for_r']).sum()
#r = r.reset_index(level='source_for_r')
r.index = pd.PeriodIndex(r.index, freq='D')
r = r.loc[r.index <= pd.Period(date.today()-timedelta(days=14), freq='D')]
rivm = rivm.merge(r, on='Date', how='outer')

homes_for_elderly.rename(columns={
    'Date_of_statistic_reported': 'Date',
    'Total_cases_reported': 'Positive_tests_elderly_homes',
    'Total_deceased_reported': 'Deceased_elderly_homes',
    'Total_new_infected_locations_reported': 'new_infected_elderly_homes',
    'Total_infected_locations_reported': 'balance_infected_elderly_homes'
    }, inplace=True)
homes_for_elderly = homes_for_elderly.groupby('Date').sum()
homes_for_elderly.index = pd.PeriodIndex(homes_for_elderly.index, freq='D')
homes_for_elderly = homes_for_elderly.loc['2020':]
rivm = rivm.merge(homes_for_elderly, on='Date', how='outer')

## Not yet implemented
# homes_for_disabled
# seventy_plus_living_at_home
# sewage
# support_and_behavior
# general_practioner

## STEP FOR ADDING
# rename columns if nececary
# cleanup if nececary
# groupby('Date').sum()
# re-order columns in nececeray
# apply periodIndex(df, freq='D')

#rivm = rivm.astype(int)
print('dataframe for RIVM data created: rivm')
rivm

dataframe for RIVM data created: rivm


,prev_low,prev_avg,prev_up,Deceased_covid,Positive_tests,Hospital_admission,Hospital_admission_NICE,Hospital_admission_NICE_notification,IC_admission,IC_admission_notification,...,IC_Nieuwe_Opnames_COVID,Kliniek_Nieuwe_Opnames_COVID,IC_Bedden_TOTAAL,Rt_low,Rt_avg,Rt_up,Positive_tests_elderly_homes,Deceased_elderly_homes,new_infected_elderly_homes,balance_infected_elderly_homes
Date,,,,,,,,,,,,,,,,,,,,,
2020-02-17,772.0,1203.0,1631.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.50,1.93,4.00,NaN,NaN,NaN,NaN
2020-02-18,900.0,1375.0,1843.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.00,1.74,4.00,NaN,NaN,NaN,NaN
2020-02-19,1099.0,1621.0,2168.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.50,1.59,2.83,NaN,NaN,NaN,NaN
2020-02-20,1300.0,1890.0,2499.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.83,1.74,2.83,NaN,NaN,NaN,NaN
2020-02-21,1598.0,2268.0,2953.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,0.67,2.03,3.67,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-12-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,32.0,182.0,1039.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2021-12-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,39.0,211.0,1077.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-02-03,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [20]:
rivm_per_week = rivm
rivm_per_week['YW'] = rivm_per_week.index.year*100+rivm_per_week.index.strftime('%W').astype(int)
rivm_per_week = rivm_per_week.groupby(by='YW').sum()
rivm_per_week = rivm_per_week[rivm_per_week.index < deceased.index.max()]

print('dataframe for RIVM data created: rivm_per_week')
rivm_per_week

dataframe for RIVM data created: rivm_per_week


,prev_low,prev_avg,prev_up,Deceased_covid,Positive_tests,Hospital_admission,Hospital_admission_NICE,Hospital_admission_NICE_notification,IC_admission,IC_admission_notification,...,IC_Nieuwe_Opnames_COVID,Kliniek_Nieuwe_Opnames_COVID,IC_Bedden_TOTAAL,Rt_low,Rt_avg,Rt_up,Positive_tests_elderly_homes,Deceased_elderly_homes,new_infected_elderly_homes,balance_infected_elderly_homes
YW,,,,,,,,,,,,,,,,,,,,,
202000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,1161.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0
202001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,78.0,424.0,5234.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0
202005,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,3589.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0
202006,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,72.0,381.0,2762.0,0.00,0.00,0.00,0.0,0.0,0.0,0.0
202007,9960.0,14217.0,18553.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,3.67,13.38,25.66,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202107,481643.0,663766.0,847875.0,407.0,27378.0,452.0,1221.0,1217.0,240.0,245.0,...,195.0,1213.0,7203.0,7.13,7.34,7.55,747.0,210.0,66.0,3372.0
202108,483669.0,662276.0,843164.0,346.0,32033.0,427.0,1213.0,1367.0,229.0,225.0,...,192.0,1199.0,7017.0,6.88,7.09,7.30,503.0,123.0,50.0,3034.0
202109,521577.0,706080.0,890029.0,268.0,31373.0,430.0,1236.0,1241.0,279.0,276.0,...,157.0,845.0,4301.0,7.62,7.83,8.04,325.0,88.0,35.0,2552.0


### CBS
### Deceased

In [21]:
# Totals only
if 'ID' in deceased.columns:
    deceased = deceased.drop(columns='ID')
if 'Geslacht' in deceased.columns:
    deceased = deceased.loc[deceased['Geslacht'] == 'Totaal mannen en vrouwen',:]
    deceased.drop(columns='Geslacht', inplace=True)
deceased = deceased.loc[deceased['LeeftijdOp31December'] != 'Totaal leeftijd',:]
# weeks only, no totals for year
deceased = deceased.loc[deceased.Perioden.str.len() != 4]
deceased['year'] = deceased.loc[:,'Perioden'].str.slice(stop=4)
deceased.year = deceased['year'].astype(int)
# split Perioden into year and week
deceased['week'] = deceased.loc[:,'Perioden'].str.slice(start=10,stop=12)
deceased = deceased.loc[deceased.week != '']
deceased['week'] = deceased['week'].astype(int)
deceased['tmp_1'] = deceased.Perioden.str.slice(start=13,stop=14)
deceased['tmp_2'] = deceased.Perioden.str.slice(start=14,stop=15)
deceased['dagen_week'] = deceased['tmp_2'].combine(deceased['tmp_1'], max)
deceased = deceased.drop(columns='tmp_1')
deceased = deceased.drop(columns='tmp_2')
deceased['dagen_week'].replace('',np.nan,inplace=True)
deceased['dagen_week'] = deceased['dagen_week'].astype(float)
# set Index
deceased['YW'] = deceased['year']*100+deceased['week']
deceased['YW'] = deceased['YW'].astype(int)
deceased.set_index(deceased['YW'], inplace=True)
if 'YW' in deceased.columns:
    deceased.drop(columns='YW', inplace=True)
#deceased.reset_index(inplace=True, drop=True)
deceased = deceased.rename(columns={'Overledenen_1': 'Deceased'})

print('dataframe from cbs created, per week: deceased_per_week')
deceased

ValueError: The truth value of a Series is ambiguous. Use a.empty, a.bool(), a.item(), a.any() or a.all().

##### Merge per week cbs decease with rivm_per_week

In [22]:
deceased.merge(rivm_per_week, on='YW', how='outer')

,LeeftijdOp31December,Perioden,Deceased,year,week,dagen_week,tmp_1,tmp_2,prev_low,prev_avg,...,IC_Nieuwe_Opnames_COVID,Kliniek_Nieuwe_Opnames_COVID,IC_Bedden_TOTAAL,Rt_low,Rt_avg,Rt_up,Positive_tests_elderly_homes,Deceased_elderly_homes,new_infected_elderly_homes,balance_infected_elderly_homes
YW,,,,,,,,,,,,,,,,,,,,,
199500,0 tot 65 jaar,1995 week 0 (1 dag),62.0,1995.0,0.0,1.0,1,,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199500,65 tot 80 jaar,1995 week 0 (1 dag),136.0,1995.0,0.0,1.0,1,,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199500,80 jaar of ouder,1995 week 0 (1 dag),196.0,1995.0,0.0,1.0,1,,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199501,0 tot 65 jaar,1995 week 1,476.0,1995.0,1.0,NaN,,,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
199501,65 tot 80 jaar,1995 week 1,901.0,1995.0,1.0,NaN,,,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
202111,80 jaar of ouder,2021 week 11,1632.0,2021.0,11.0,NaN,,,666125.0,889929.0,...,280.0,1500.0,7300.0,7.28,4.25,7.62,213.0,34.0,34.0,1863.0
202112,0 tot 65 jaar,2021 week 12,399.0,2021.0,12.0,NaN,,,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
202112,65 tot 80 jaar,2021 week 12,1005.0,2021.0,12.0,NaN,,,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


#### population

In [ ]:
# dropping unnececary columns
if 'ID' in population.columns:
    population.drop(columns=['ID','BevolkingAanHetBeginVanDePeriode_1','TotaleVestiging_4',
       'VestigingVanuitEenAndereGemeente_5', 'Immigratie_6',
       'TotaalVertrekInclAdmCorrecties_7', 'VertrekNaarAndereGemeente_8',
       'EmigratieInclusiefAdmCorrecties_9', 'OverigeCorrecties_10',
       'Bevolkingsgroei_11', 'BevolkingsgroeiRelatief_12',
       'BevolkingsgroeiSinds1Januari_13',
       'BevolkingsgroeiSinds1JanuariRela_14'], inplace=True)
population = population.rename(columns={"LevendGeborenKinderen_2": "Born", "Overledenen_3": "Deceased", "BevolkingAanHetEindeVanDePeriode_15": "Population"})
# Setting PeriodIndex
population['year'] = population.loc[:,'Perioden'].str.slice(stop=4)
population['month'] = population.loc[:,'Perioden'].str.slice(start=5)
population = population.loc[population['month'] != '',:]
population['month'] = population.loc[:,'month'].replace({
    'januari': '01',
    'februari': '02',
    'maart': '03',
    'april': '04',
    'mei': '05',
    'juni': '06',
    'juli': '07',
    'augustus': '08',
    'september': '09',
    'oktober': '10',
    'november': '11',
    'december': '12'})
population = population.set_index(population['year'] + '-' + population['month'])
population.index = pd.PeriodIndex(population.index, freq='M')

print('dataframe from cbs created, per month: population')
population